In [36]:
import pandas as pd
import re
import numpy as np
from scipy import stats
from sklearn import metrics
from bayesian_player_ratings import *
from openskill.models import PlackettLuce, BradleyTerryFull
import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.precision', 4)

import trueskillthroughtime as ttt

In [2]:
match_summary = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/match_summary.csv")
match_summary = score_col_splitter(match_summary, "Q4_Score")
match_summary['Season'] = match_summary['Match_ID'].apply(lambda x: int(x[:4]))

player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/scored_player_stats.csv")
player_stats = create_additional_player_stats_variables(player_stats)
player_stats['exp_vaep_value'] = player_stats['exp_vaep_value'].fillna(0)
player_stats['exp_offensive_value'] = player_stats['exp_offensive_value'].fillna(0)
player_stats['exp_defensive_value'] = player_stats['exp_defensive_value'].fillna(0)

match_stats = aggregate_player_to_match_stats(player_stats)

match_summary_stats = match_summary.merge(match_stats, how = "left", on = "Match_ID")

In [3]:
player_stats[player_stats['Match_ID'] == '202101_BrisbaneLions_Sydney'].tail()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
41,202101_BrisbaneLions_Sydney,Sydney,Sam Wicks,202101,Sam_Wicks,MEDIUM_FORWARD,CD_I1006232,22,180,80,45,RIGHT,NSW,2018.0,2020.0,Manly-Warringah (NSW)/Sydney (NEAFL),NaN,other,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-09-14,85.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,41.7,5.0,1.0,0.0,0.0,2.0,75.0,12.0,69.0,9.0,5.0,1.0,0.0,1.0,100.0,0.0,2.0,2.0,6.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,83.3,0.0,0.0,6.0,1.0,2.0,0.0,0.0,194.0,1.0,15.0,14.2,0.0,0.0,5.0,1.0,2.0,0.0,0.0,5.0,1.0,0.0,12.0,1.0,7.0,NaN,NaN,Half Forward Flank Left,FINAL_TEAM,Half-Forward,Forward,2021.0,0.0,2021,5.0964,0.0739,-0.0076,1.4309,1.5528,-0.1219,1.8600,2.6695,-0.8095,0.2936,-0.0012,0.1368,1.9963,Sam Wicks_2021,12.0,6.9036,Brisbane Lions,Sydney,Brisbane Lions,1,01,20211
42,202101_BrisbaneLions_Sydney,Sydney,Tom Hickey,202101,Tom_Hickey,RUCK,CD_I294266,30,202,96,31,LEFT,QLD,2010.0,2011.0,Morningside (Qld)/Gold Coast/St Kilda/West Coast,NaN,zone,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1991-03-06,82.0,0.0,0.0,31.0,5.0,3.0,0.0,0.0,3.0,0.0,60.9,14.0,1.0,0.0,0.0,7.0,82.6,23.0,92.0,19.0,10.0,0.0,2.0,3.0,0.0,1.0,0.0,9.0,11.0,24.0,6.0,25.0,42.9,5.0,1.0,5.0,83.3,0.0,0.0,12.0,1.1,3.0,0.0,0.0,400.0,6.0,11.0,19.0,2.0,56.0,7.0,3.0,0.0,5.0,4.0,1.0,0.0,9.0,23.0,6.0,9.0,Tom_Hickey,7.0,Ruck,FINAL_TEAM,Ruck,Ruck,2021.0,1.0,2021,0.0000,-0.4230,0.0300,2.3603,1.5111,0.8492,15.2474,10.0000,5.2475,-0.3704,0.0351,1.2706,9.7125,Tom Hickey_2021,0.0,0.0000,Brisbane Lions,Sydney,Brisbane Lions,1,01,20211
43,202101_BrisbaneLions_Sydney,Sydney,Tom McCartin,202101,Tom_McCartin,KEY_DEFENDER,CD_I1008198,22,193,95,30,RIGHT,VIC,2017.0,2018.0,"St Joseph&apos;s, Geel (Vic)/St Joseph&apos;s ...",33.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-12-30,87.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,1.0,60.0,6.0,0.0,0.0,0.0,5.0,72.7,11.0,49.0,8.0,7.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,1.0,3.0,9.0,77.8,0.0,0.0,9.0,4.5,4.0,0.0,0.0,157.0,4.0,5.0,8.0,1.0,0.0,4.0,3.0,0.0,4.0,0.0,1.0,0.0,0.0,10.0,3.0,4.0,NaN,NaN,Full Back,FINAL_TEAM,Key-Back,Back,2021.0,0.0,2021,0.0000,-0.1081,0.0234,0.7347,0.1729,0.5618,4.8127,3.6471,1.1657,-0.4397,0.1014,0.5339,2.5907,Tom McCartin_2021,0.0,0.0000,Brisbane Lions,Sydney,Brisbane Lions,1,01,20211
44,202101_BrisbaneLions_Sydney,Sydney,Tom Papley,202101,Tom_Papley,MEDIUM_FORWARD,CD_I996765,25,177,79,11,RIGHT,VIC,2016.0,2016.0,Bunyip (Vic)/Gippsl

OpenSkill

In [5]:
model = PlackettLuce()

In [8]:
dayne_zorko = model.rating(name = "zorko", mu = 0)
print(dayne_zorko)

Plackett-Luce Player Data: 

id: 4b21855afa5f44aab95ca99409d0fbfe
name: zorko
mu: 0
sigma: 8.333333333333334



In [9]:
match_rating = player_stats[(player_stats['Match_ID'] == '202101_BrisbaneLions_Sydney') & (player_stats['Player'] == "Dayne Zorko")]['exp_vaep_value'].iloc[0]
dayne_zorko_match = model.rating(mu = match_rating)

In [15]:
match = [[dayne_zorko], [dayne_zorko_match]]
match

[[PlackettLuceRating(mu=0, sigma=8.333333333333334)],
 [PlackettLuceRating(mu=11.6167738674, sigma=8.333333333333334)]]

In [16]:
model.rate(match)

[[PlackettLuceRating(mu=3.7272680445145245, sigma=8.112507988314318)],
 [PlackettLuceRating(mu=7.889505822885475, sigma=8.112507988314318)]]

Dayne Zorko

In [141]:
initial_mu = 0
initial_sigma = 20
sigma = 15
initial_beta = 30
beta = 50

In [142]:
model = PlackettLuce(mu = initial_mu, sigma = initial_sigma, beta = initial_beta)

In [143]:
zorko_ratings = player_stats[(player_stats['Player'] == "Dayne Zorko")]
zorko_ratings.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
4,202101_BrisbaneLions_Sydney,Brisbane Lions,Dayne Zorko,202101,Dayne_Zorko,MIDFIELDER,CD_I261224,32,175,78,15,RIGHT,QLD,2011.0,2012.0,Surfers Paradise (Qld)/Broadbeach (Qld),NaN,other,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1989-02-09,82.0,2.0,0.0,10.0,2.0,7.0,0.0,0.0,0.0,0.0,42.1,8.0,0.0,0.0,0.0,8.0,68.2,22.0,92.0,15.0,10.0,1.0,6.0,1.0,33.3,0.0,1.0,7.0,7.0,0.0,0.0,0.0,0.0,6.0,1.0,4.0,66.7,0.0,0.0,15.0,2.1,6.0,1.0,1.0,445.0,1.0,20.0,14.6,2.0,0.0,7.0,3.0,3.0,1.0,2.0,6.0,2.0,4.0,19.0,5.0,11.0,Dayne_Zorko,2.0,Centre,FINAL_TEAM,Centre,Midfield,2021.0,0.0,2021,7.9532,-0.0967,0.0278,1.3032,1.0745,0.2287,11.6168,8.7870,2.8298,-0.2753,0.0961,0.5303,6.2803,Dayne Zorko_2021,8.0,0.0468,Brisbane Lions,Sydney,Sydney,1,01,20211
464,202102_Geelong_BrisbaneLions,Brisbane Lions,Dayne Zorko,202102,Dayne_Zorko,MIDFIELDER,CD_I261224,32,175,78,15,RIGHT,QLD,2011.0,2012.0,Surfers Paradise (Qld)/Broadbeach (Qld),NaN,other,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1989-02-09,82.0,1.0,0.0,13.0,1.0,8.0,0.0,0.0,1.0,0.0,38.1,8.0,0.0,0.0,0.0,9.0,54.5,22.0,76.0,12.0,11.0,2.0,4.0,0.0,0.0,3.0,0.0,7.0,2.0,0.0,0.0,0.0,0.0,9.0,0.0,3.0,55.0,0.0,0.0,20.0,10.0,5.0,0.0,0.0,612.0,2.0,23.0,6.9,1.0,0.0,6.0,3.0,1.0,1.0,5.0,2.0,0.0,6.0,21.0,5.0,13.0,NaN,NaN,Centre,FINAL_TEAM,Centre,Midfield,2021.0,0.0,2021,2.7342,0.4707,0.3471,1.2826,1.1902,0.0923,8.0030,9.3013,-1.2983,0.2505,0.0760,0.7826,7.9955,Dayne Zorko_2021,1.0,-1.7342,Geelong,Brisbane Lions,Geelong,2,02,20212
879,202103_Collingwood_BrisbaneLions,Brisbane Lions,Dayne Zorko,202103,Dayne_Zorko,MIDFIELDER,CD_I261224,32,175,78,15,RIGHT,QLD,2011.0,2012.0,Surfers Paradise (Qld)/Broadbeach (Qld),NaN,other,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1989-02-09,81.0,1.0,0.0,10.0,0.0,3.0,0.0,0.0,0.0,0.0,21.7,5.0,0.0,0.0,0.0,10.0,70.8,24.0,133.0,17.0,15.0,0.0,1.0,1.0,0.0,2.0,0.0,5.0,3.0,0.0,0.0,0.0,0.0,9.0,0.0,4.0,71.4,0.0,0.0,21.0,7.0,11.0,0.0,0.0,574.0,0.0,25.0,11.8,4.0,0.0,7.0,1.0,2.0,0.0,0.0,8.0,2.0,0.0,23.0,3.0,18.0,NaN,NaN,Wing Left,FINAL_TEAM,Wing,Midfield,2021.0,0.0,2021,5.0304,0.1524,0.0000,0.9126,0.9058,0.0068,8.5494,8.0771,0.4723,-0.2396,-0.0039,1.1454,9.2568,Dayne Zorko_2021,1.0,-4.0304,Collingwood,Brisbane Lions,Collingwood,3,03,20213
1570,202104_WesternBulldogs_BrisbaneLions,Brisbane Lions,Dayne Zorko,202104,Dayne_Zorko,MIDFIELDER,CD_I261224,32,175,78,15,RIGHT,QLD,2011.0,2012.0,Surfers Par

In [144]:
dayne_zorko = model.rating()

counter = 0
for index, row in zorko_ratings.iterrows():
    print('Before match rating: \t{:.3}, {:.3}'.format(dayne_zorko.mu, dayne_zorko.sigma))
    print(model.beta)
    if row['exp_vaep_value'] < 0:
        new_rating =  0 
    else:
        new_rating = row['exp_vaep_value']
    match_rating = model.rating(mu = new_rating)
    print('Match rating: \t\t{}'.format(new_rating))
    
    if dayne_zorko.mu > new_rating:
        match = [[match_rating], [dayne_zorko]]
    else:
        match = [[dayne_zorko], [match_rating]]
    [[dayne_zorko], _] = model.rate(match)
    print('After match rating: \t{:.3}'.format(dayne_zorko.mu))
    
    if counter > 20:
        model = PlackettLuce(mu = initial_mu, sigma = sigma, beta = beta)
    else:
        model = PlackettLuce(mu = initial_mu, sigma = initial_sigma, beta = initial_beta)
    counter += 1

Before match rating: 	0.0, 20.0
30
Match rating: 		11.6167738674
After match rating: 	4.37
Before match rating: 	4.37, 19.9
30
Match rating: 		8.0029727265
After match rating: 	8.37
Before match rating: 	8.37, 19.7
30
Match rating: 		8.549419652
After match rating: 	12.2
Before match rating: 	12.2, 19.6
30
Match rating: 		4.4194708324
After match rating: 	8.66
Before match rating: 	8.66, 19.8
30
Match rating: 		5.103083463
After match rating: 	9.17
Before match rating: 	9.17, 19.8
30
Match rating: 		14.7763852397
After match rating: 	13.2
Before match rating: 	13.2, 19.7
30
Match rating: 		4.879096355999999
After match rating: 	9.13
Before match rating: 	9.13, 19.8
30
Match rating: 		11.098732043
After match rating: 	13.1
Before match rating: 	13.1, 19.7
30
Match rating: 		15.7888691167
After match rating: 	17.0
Before match rating: 	17.0, 19.6
30
Match rating: 		11.450110382
After match rating: 	15.6
Before match rating: 	15.6, 19.8
30
Match rating: 		14.7584929701
After match rating: